In [171]:
import pandas as pd
import numpy as np
from dateutil import parser

<h2> Reading the dataset </h2>

In [172]:
# Reading other dataset
Data1 = pd.read_excel('/Users/briankimanzi/Documents/programming Languages/PythonProgramming/JupyterNoteBook/Datasets/MpesaStatement_Katra.xlsx')

<h2> Cleaning the dataset </h2>

In [173]:
Data1['Completion Time'] = pd.to_datetime(Data1['Completion Time'])

In [174]:
# Anonymising the dataset 
def transaction(x):
    x = str(x).strip()
    if x.startswith('Merchant Payment'):
        index = x.find(' - ')+2
        name = x[index:].strip().upper()
        
        return 'BUY GOODS', name

    elif x.startswith('Deposit of Funds'):
        index = x.find(' - ')+2
        name = x[index:].strip().upper()
        
        return 'AGENT DEPOSIT', name

    elif x.startswith('OD Loan Repayment'):
        return 'FULIZA REPAYMENT', 'FULIZA'

    elif x.startswith('OverDraft of Credit Party'):
        return 'FULIZA TAKEN', 'FULUZA'

    elif x.startswith('M-Shwari Deposit'):
        return 'M-SHWARI DEPOSIT FROM M-PESA', 'M-SHWARI'
        
    elif x.startswith('KCB M-PESA Deposit'):
        return 'KCB M-PESA DEPOSIT FROM M-PESA', 'KCB DEPOSIT'

    elif x.startswith('KCB M-PESA Withdraw'):
        return 'KCB M-PESA WITHDRAW FROM M-PESA', 'KCB WITHDRAW'

    elif x.startswith('Customer Transfer'):
        index = x.find(' - ')
        to_search = x[index:]
        last = 1

        for i in range(len(to_search)):
            try:
                int(to_search[i])
                last = i
                
            except:
                v = 10

        name = x[index+last+2:].strip().upper()
        return 'SEND MONEY', name

    elif x.startswith('M-Shwari Withdraw'):
        
        return 'M-SHWARI WITHDRAW FROM M-PESA', 'M-SHWARI WITHDRAW'

    elif x.startswith('Pay Bill Charge'):
        # if x.strip() == 'Pay Bill Charge':
            return 'PAY BILL CHARGES', 'TRANSACTION COST'
        
    elif x.startswith('Pay Bill Online to'):
        if x == 'Pay Bill Online to':  
            return 'PAY BILL ONLINE', 'TRANSACTION COST'
        else:
            start = len('Pay Bill Online to')+8
            end = x.lower().find('acc')
            name = x[start:end].strip().upper()
            return 'PAY BILL', name
    elif x.startswith('Pay Bill'):
        if x == 'Pay Bill':  
            return 'PAY BILL', 'UNKNOWN'
        else:
            start = x.find(' - ') + 3 if ' - ' in x else len('Pay Bill') + 1
            end = x.lower().find('acc')
            name = x[start:end].strip().upper() if end != -1 else x[start:].strip().upper()
            return 'PAY BILL', name

    elif x.startswith('Funds received'):
        index = x.find(' - ')
        to_search = x[index:]
        last = 1

        for i in range(len(to_search)):
            try:
                int(to_search[i])
                last = i
            except:
                v = 10

        name = x[index+last+2:].strip().upper()
        
        return 'RECEIVED FUNDS', name

    elif x.startswith('Customer Payment to Small Business') or x.startswith('Customer Send Money'):
        index = x.find(' - ')
        to_search = x[index:]
        last = 1

        for i in range(len(to_search)):
            try:
                int(to_search[i])
                last = i
            except:
                v = 10

        name = x[index+last+2:].strip().upper()
        
        return 'POCHI LA BIASHARA', name

    elif x.startswith('Airtime Purchase'):
        return 'AIRTIME PURCHASE', 'AIRTIME'

    elif x.startswith('Business Payment From'):
        index = x.find(' - ')
        end = x.lower().find('via')
        to_search = x[index:]
        last = 1
        name = x[index:end].strip().upper()
        return 'FUNDS RECEIVED FROM BUSINESS', name

    elif x.startswith('Customer Transfer of Funds Charge'):
        return 'TRANSACTION COST', 'TRANSACTION COST'
    
    elif x.startswith('Buy Bundles Online'):
        return 'BUNDLES PURCHASE', 'BUNDES PURCHASE'
    
    elif x.startswith('Customer Withdrawal'):
        index = x.find(' - ')+2
        name = x[index:].strip().upper()
        return 'CASH WITHDRAWAL', name
    
    elif x.startswith('Withdrawal Charge'):
        return 'CASH WITHDRAWAL CHARGES', "TRANSACTION COST"

    elif x.startswith('Promotion Payment'):
        return 'PROMOTION PAYMENT', 'M-PESA OFFERS. VIA API'
    
    elif x.startswith('Business Payment'):
        return 'BUSINESS PAYMENT', 'CO-OP BANK VIA API'
        
    elif x.startswith('Savings Contribution'):
        return 'TO HUSTLER FUND SAVINGS', 'HUSTLER FUND'
    
    elif x.startswith('Term Loan Disbursement for H- Fund') or x.startswith('Term Loan Disbursement for H-Fund'):
        return 'HUSTLER FUND Disbursement'.upper(), 'HUSTLER FUND'
    
    elif x.startswith('Term Loan Repayment for H- Fund') or x.startswith('Term Loan Repayment for H-Fund'):
        return 'HUSTLER FUND REPAYMENT', 'HUSTLER FUND'
    
    else:
        return 'UNIDENTIFIED', 'UNIDENTIFIED'

In [175]:
Data1['Details'] = Data1['Details'].apply(lambda x: x.replace('\r', ' '))

In [176]:
details = list(Data1['Details'].apply(lambda x: transaction(x)).values)
transactionType = [i[0] for i in details]
TransactionParty = [i[1] for i in details]

In [177]:
# changing the date format
def change_date(date):
    date_str = str(date)
    date = parser.parse(date_str)
    return (date.year, date.month, date.day, date.weekday(),date.hour, date.minute, date.second)

In [178]:
date = Data1['Completion Time'].apply(lambda x: change_date(x)).values
Year = [i[0] for i in date]
Month = [i[1] for i in date]
Date = [i[2] for i in date]
Weekday = [i[3] for i in date]
Hour = [i[4] for i in date]
Minute = [i[5] for i in date]
Seconds = [i[6] for i in date]

transactionDay = []

reverseDay = Date.copy()[::-1]
datey = reverseDay[0]

d = 1

for i in reverseDay:
    if i == datey:
        transactionDay.append(d)
    else:
        datey = i
        d += 1
        transactionDay.append(d)

transactionDay.reverse()

In [179]:
receipt = list(Data1['Receipt No.'].values)

In [180]:
paid_in = Data1['Paid In'].fillna('NAN_VALUE').values

In [181]:
def withdrawAmount(x):
    try:
        if x < 0:
            return -x
        else:
            return x
    except:
        return x

In [182]:
withdraw = list(Data1['Withdrawn'].apply(lambda x: withdrawAmount(x)).values)
withdraw                

[14.0,
 3699.0,
 10.0,
 1000.0,
 5.0,
 53.0,
 460.0,
 10.0,
 646.0,
 2400.0,
 29.0,
 2000.0,
 nan,
 nan,
 300.0,
 5.0,
 500.0,
 1000.0,
 1000.0,
 nan,
 160.0,
 720.0,
 10.0,
 300.0,
 14800.0,
 115.0,
 8000.0,
 164.0,
 10.0,
 1000.0,
 200.0,
 173.0,
 5.0,
 20.0,
 nan,
 11.0,
 5.0,
 173.0,
 20.0,
 5.0,
 94.0,
 3000.0,
 nan,
 200.0,
 15.0,
 nan,
 200.0,
 1000.0,
 55.0,
 nan,
 2775.0,
 nan,
 30.0,
 20.0,
 10.0,
 1000.0,
 nan,
 20.0,
 20.0,
 20.0,
 1000.0,
 nan,
 5.0,
 31.0,
 40.0,
 20.0,
 25.0,
 1000.0,
 30.0,
 20.0,
 1000.0,
 3699.0,
 nan,
 10.0,
 nan,
 1600.0,
 nan,
 5.0,
 530.0,
 27.0,
 20.0,
 29.0,
 2200.0,
 nan,
 1200.0,
 nan,
 8.0,
 3699.0,
 nan,
 5.0,
 5.0,
 10.0,
 70.0,
 10.0,
 1000.0,
 830.0,
 nan,
 29.0,
 nan,
 14.0,
 30.0,
 5000.0,
 nan,
 29.0,
 nan,
 5.0,
 450.0,
 nan,
 42.0,
 4686.0,
 30.0,
 nan,
 nan,
 3699.0,
 nan,
 nan,
 7500.0,
 530.0,
 10.0,
 1000.0,
 nan,
 nan,
 1600.0,
 500.0,
 nan,
 29.0,
 290.0,
 3000.0,
 nan,
 3200.0,
 5.0,
 500.0,
 nan,
 nan,
 10.0]

In [183]:
transaction_Data = []

for i in range(len(paid_in)):
    try:
        x = float(paid_in[i])
        transaction_Data.append((x, 'PAID IN'))
    except:
        x = float(withdraw[i])
        transaction_Data.append((x, 'WITHDRAW'))

In [184]:
transaction_amount = [i[0] for i in transaction_Data]
paid_in_or_withdraw = [i[1] for i in transaction_Data]

In [185]:
balance =list(Data1["Balance"].values)

In [188]:
katra = pd.DataFrame({
    "Receipt":receipt,
    "transaction_Day": transactionDay,
    "Year" : Year,
    "Month" : Month,
    "Date" : Date,
    "Weekday" : Weekday,
    "Hour" : Hour,
    "Minute" : Minute,
    "Seconds" : Seconds,
    "Transaction_type" : transactionType,
    "Transaction_party" : TransactionParty,
    "Transaction_amount" : transaction_amount,
    "paid_in_or_Withdraw" : paid_in_or_withdraw,
    "Balance" : balance
})
katra.set_index('Receipt', inplace=True)

In [170]:
katra.to_csv('/Users/briankimanzi/Documents/programming Languages/PythonProgramming/JupyterNoteBook/Datasets/katra.csv')

In [189]:
katra

,transaction_Day,Year,Month,Date,Weekday,Hour,Minute,Seconds,Transaction_type,Transaction_party,Transaction_amount,paid_in_or_Withdraw,Balance
Receipt,,,,,,,,,,,,,
TD71R8CU5T,68,2025,4,7,0,12,24,25,AIRTIME PURCHASE,AIRTIME,14.0,WITHDRAW,0.71
TD48BNDSIK,67,2025,4,4,4,5,10,29,PAY BILL,SAFARICOM HOME,3699.0,WITHDRAW,96822.71
TCU0NDJPKK,66,2025,3,30,6,7,13,23,PAY BILL CHARGES,TRANSACTION COST,10.0,WITHDRAW,27.71
TCU0NDJPKK,66,2025,3,30,6,7,13,23,PAY BILL,KPLC PREPAID,1000.0,WITHDRAW,37.71
TCS0HGRZ5U,65,2025,3,28,4,20,9,27,AIRTIME PURCHASE,AIRTIME,5.0,WITHDRAW,1037.71
TCS0HGJXSC,65,2025,3,28,4,20,8,25,AIRTIME PURCHASE,AIRTIME,53.0,WITHDRAW,1042.71
TCP344ZT83,64,2025,3,25,1,20,58,18,AIRTIME PURCHASE,AIRTIME,460.0,WITHDRAW,1095.71
TCM0NPVBU0,63,2025,3,22,5,12,57,46,PAY BILL CHARGES,TRANSACTION COST,10.0,WITHDRAW,1555.71
TCM0NPVBU0,63,2025,3,22,5,12,57,46,PAY BILL,PAYSTACK PAYMENTS KENYA LIMITED,646.0,WITHDRAW,1565.71
